In [22]:
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
from datetime import datetime
import zipfile
import os

BASE_URL = "https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/"
DATA_DIR = "data"

html = urlopen(BASE_URL).read()
soup = BeautifulSoup(html, "html.parser")

ano_atual = datetime.now().year
anos = []

for link in soup.find_all("a"):
    href = link.get("href")
    if href and href.endswith("/") and href[:4].isdigit():
        ano = int(href[:4])
        if ano <= ano_atual:
            anos.append(ano)

ano_mais_recente = max(anos)
URL_ANO = f"{BASE_URL}{ano_mais_recente}/"

print(f"Ano mais recente: {ano_mais_recente}")
print(f"URL selecionada: {URL_ANO}")

os.makedirs(DATA_DIR, exist_ok=True)

ANO_DIR = os.path.join(DATA_DIR, str(ano_mais_recente))
os.makedirs(ANO_DIR, exist_ok=True)

html_ano = urlopen(URL_ANO).read()
soup_ano = BeautifulSoup(html_ano, "html.parser")

zips = []
for link in soup_ano.find_all("a"):
    href = link.get("href")
    if href and href.lower().endswith(".zip"):
        zips.append(href)

print("\nZIPs encontrados:")
for z in zips:
    print(" -", z)

for z in zips:
    zip_url = URL_ANO + z
    zip_path = os.path.join(ANO_DIR, z)
    extract_folder = os.path.join(ANO_DIR, z.replace('.zip', ''))

    if not os.path.exists(zip_path):
        print(f"\nBaixando: {z}")
        urlretrieve(zip_url, zip_path)
    else:
        print(f"\nArquivo ZIP já existe: {z}")

    if not os.path.exists(extract_folder):
        print(f"Extraindo: {z} para {extract_folder}")
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_folder)
    else:
        print(f"Conteúdo de {z} já foi extraído anteriormente.")

Ano mais recente: 2025
URL selecionada: https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/2025/

ZIPs encontrados:
 - 1T2025.zip
 - 2T2025.zip
 - 3T2025.zip

Arquivo ZIP já existe: 1T2025.zip
Conteúdo de 1T2025.zip já foi extraído anteriormente.

Arquivo ZIP já existe: 2T2025.zip
Conteúdo de 2T2025.zip já foi extraído anteriormente.

Arquivo ZIP já existe: 3T2025.zip
Conteúdo de 3T2025.zip já foi extraído anteriormente.


In [23]:
import glob

print("Validando arquivos extraídos...")
pastas_extracao = glob.glob(os.path.join(ANO_DIR, "*/"))

for pasta in pastas_extracao:
    csvs = glob.glob(os.path.join(pasta, "*.csv"))
    if len(csvs) > 0:
        print(f"Sucesso: {os.path.basename(os.path.normpath(pasta))} contém {len(csvs)} arquivo(s) CSV.")
    else:
        print(f"Erro: A pasta {pasta} está vazia ou não contém CSVs.")

Validando arquivos extraídos...
Sucesso: 1T2025 contém 1 arquivo(s) CSV.
Sucesso: 2T2025 contém 1 arquivo(s) CSV.
Sucesso: 3T2025 contém 1 arquivo(s) CSV.


In [24]:
import pandas as pd

print("Verificando encoding dos arquivos...")
for pasta in pastas_extracao:
    csv_file = glob.glob(os.path.join(pasta, "*.csv"))[0]
    try:
        
        pd.read_csv(csv_file, sep=None, engine='python', nrows=5, encoding='utf-8')
        print(f"✅ {os.path.basename(csv_file)}: UTF-8")
    except:
        try:
        
            pd.read_csv(csv_file, sep=None, engine='python', nrows=5, encoding='latin-1')
            print(f"{os.path.basename(csv_file)}: ISO-8859-1 (Latin-1)")
        except Exception as e:
            print(f"Erro ao detectar encoding de {csv_file}: {e}")

Verificando encoding dos arquivos...
✅ 1T2025.csv: UTF-8
✅ 2T2025.csv: UTF-8
✅ 3T2025.csv: UTF-8


In [25]:
lista_para_merge = []
termo_gatilho = "EVENTOS/ SINISTROS CONHECIDOS OU AVISADOS"

caminhos_csv = glob.glob(os.path.join(ANO_DIR, "**/*.csv"), recursive=True)

for caminho in caminhos_csv:
    if "output" in caminho:
        continue
        
    nome = os.path.basename(caminho)
    df_individual = pd.read_csv(caminho, sep=';', encoding='latin-1', low_memory=False)
    
    if df_individual['DESCRICAO'].astype(str).str.contains(termo_gatilho, case=False).any():
        print(f"Incluido: {nome}")
        lista_para_merge.append(df_individual)
    else:
        print(f"Ignorado: {nome}")

if lista_para_merge:
    df_consolidado = pd.concat(lista_para_merge, ignore_index=True)
    print(f"Linhas no merge: {len(df_consolidado)}")
else:
    df_consolidado = pd.DataFrame()
    print("Nenhum arquivo validado.")

Incluido: 1T2025.csv
Incluido: 2T2025.csv
Incluido: 3T2025.csv
Linhas no merge: 2113924


In [26]:
BASE_OUTPUT = "output"
SUBPASTA_CONSOLIDADO = os.path.join(BASE_OUTPUT, "massa_dados_trimestrais")
os.makedirs(SUBPASTA_CONSOLIDADO, exist_ok=True)

final_csv = os.path.join(SUBPASTA_CONSOLIDADO, "dados_contabeis_unificados.csv")
final_zip = os.path.join(SUBPASTA_CONSOLIDADO, "dados_contabeis_unificados.zip")

if os.path.exists(final_csv) and os.path.exists(final_zip):
    print("Arquivos ja existem. Pulando processamento.")
else:
    if not df_consolidado.empty:
        df_consolidado['DATA'] = pd.to_datetime(df_consolidado['DATA'], errors='coerce')
        df_consolidado['REG_ANS'] = df_consolidado['REG_ANS'].astype(str)
        df_consolidado['CD_CONTA_CONTABIL'] = df_consolidado['CD_CONTA_CONTABIL'].astype(str)
        
        for col in ['VL_SALDO_INICIAL', 'VL_SALDO_FINAL']:
            if df_consolidado[col].dtype == 'object':
                df_consolidado[col] = df_consolidado[col].str.replace(',', '.').astype(float)
            df_consolidado[col] = df_consolidado[col].fillna(0)

        df_consolidado['DESCRICAO'] = df_consolidado['DESCRICAO'].fillna('NAO INFORMADO').str.strip().str.upper()

        df_consolidado.to_csv(final_csv, sep=';', index=False, encoding='utf-8')
        df_consolidado.to_csv(final_zip, sep=';', index=False, encoding='utf-8', compression={'method': 'zip', 'archive_name': 'dados_contabeis_unificados.csv'})
        print(f"Arquivos gerados em: {SUBPASTA_CONSOLIDADO}")
    else:
        print("DataFrame vazio.")

Arquivos ja existem. Pulando processamento.


In [27]:
SUBPASTA_CADASTRO = os.path.join("output", "dados_cadastrais")
os.makedirs(SUBPASTA_CADASTRO, exist_ok=True)

BASE_URL_CADASTRO = "https://dadosabertos.ans.gov.br/FTP/PDA/operadoras_de_plano_de_saude_ativas/"
html_cad = urlopen(BASE_URL_CADASTRO).read()
soup_cad = BeautifulSoup(html_cad, "html.parser")

arquivo_csv = ""
for link in soup_cad.find_all("a"):
    href = link.get("href")
    if href and href.endswith(".csv"):
        arquivo_csv = href
        break

if arquivo_csv:
    url_final_cadastro = BASE_URL_CADASTRO + arquivo_csv
    caminho_destino = os.path.join(SUBPASTA_CADASTRO, arquivo_csv)
    
    if not os.path.exists(caminho_destino):
        print(f"Baixando: {arquivo_csv}")
        urlretrieve(url_final_cadastro, caminho_destino)
    else:
        print(f"Arquivo {arquivo_csv} ja existe.")
    
    df_cadastro = pd.read_csv(caminho_destino, sep=';', encoding='latin-1', dtype={'CNPJ': str, 'Registro_ANS': str})

Arquivo Relatorio_cadop.csv ja existe.
